<a href="https://colab.research.google.com/github/begumbasovali/-Predicting-Drug-Resistance-in-Mutated-EGFR/blob/main/Ceng4525_Bioinformatik_final_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 1. Copy the files from your Drive to the local Colab disk
!cp /content/drive/MyDrive/FoldX/foldx_20261231 ./foldx
!cp /content/drive/MyDrive/FoldX/rotabase.txt ./rotabase.txt

# 2. Make the file executable
!chmod +x foldx

# 3. Verify the installation
!./foldx --help

   ********************************************
   ***                                      ***
   ***             FoldX 4 (c)              ***
   ***                                      ***
   ***     code by the FoldX Consortium     ***
   ***                                      ***
   ***     Jesper Borg, Frederic Rousseau   ***
   ***    Joost Schymkowitz, Luis Serrano   ***
   ***    Peter Vanhee, Erik Verschueren    ***
   ***     Lies Baeten, Javier Delgado      ***
   ***       and Francois Stricher          ***
   *** and any other of the 9! permutations ***
   ***   based on an original concept by    ***
   ***   Raphael Guerois and Luis Serrano   ***
   ********************************************

FoldX program options:


Basic OPTIONS:
  -v [ --version ] arg (=Version beta 4)
                                        print version string
  -h [ --help ]                         produce help message
  -c [ --command ] arg                  Choose your FoldX Command:
         

In [ ]:
# v6 sürümünü indiriyoruz
!wget https://alphafold.ebi.ac.uk/files/AF-P00533-F1-model_v6.pdb -O EGFR_AlphaFold.pdb

--2026-01-07 12:51:32--  https://alphafold.ebi.ac.uk/files/AF-P00533-F1-model_v6.pdb
Resolving alphafold.ebi.ac.uk (alphafold.ebi.ac.uk)... 34.149.152.8
Connecting to alphafold.ebi.ac.uk (alphafold.ebi.ac.uk)|34.149.152.8|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘EGFR_AlphaFold.pdb’

EGFR_AlphaFold.pdb      [ <=>                ] 754.23K  --.-KB/s    in 0.04s   

2026-01-07 12:51:33 (18.6 MB/s) - ‘EGFR_AlphaFold.pdb’ saved [772334]



In [ ]:
# Bu işlem birkaç dakika sürebilir (Protein büyük olduğu için)
!./foldx --command=RepairPDB --pdb=EGFR_AlphaFold.pdb

Görüntülenen çıkış son 5000 satıra kısaltıldı.
kn electrostatic=               0.00
partial covalent interactions = 0.00
Energy_Ionisation =             2.89
Entropy Complex =               0.00
-----------------------------------------------------------
Total          = 				  471.62

Residue PROA992 has high Energy, we mutate it to itself
Repair Residue ID= PROA992

BackHbond       =               -647.21
SideHbond       =               -338.28
Energy_VdW      =               -1123.80
Electro         =               -47.93
Energy_SolvP    =               1494.63
Energy_SolvH    =               -1468.73
Energy_vdwclash =               41.71
energy_torsion  =               60.96
backbone_vdwclash=              532.20
Entropy_sidec   =               652.08
Entropy_mainc   =               1920.70
water bonds     =               0.00
helix dipole    =               -11.47
loop_entropy    =               0.00
cis_bond        =               7.31
disulfide       =               -71.41
kn ele

In [ ]:
mutations = [
    "KA28T;", "NA234D;", "VA292L;", "VA292M;", "SA306L;", "CA311F;", "QA408R;",
    "GA598V;", "MA600V;", "EA758K;", "LA858R;", "AA864V;", "SA1025N;",
    "HA1124Q;", "DA1127N;", "SA1130G;", "QA1159H;"
]

with open('individual_list_full.txt', 'w') as f:
    for m in mutations:
        f.write(m + '\n')

In [ ]:
!./foldx --command=BuildModel --pdb=EGFR_AlphaFold_Repair.pdb --mutant-file=individual_list_full.txt

   ********************************************
   ***                                      ***
   ***             FoldX 4 (c)              ***
   ***                                      ***
   ***     code by the FoldX Consortium     ***
   ***                                      ***
   ***     Jesper Borg, Frederic Rousseau   ***
   ***    Joost Schymkowitz, Luis Serrano   ***
   ***    Peter Vanhee, Erik Verschueren    ***
   ***     Lies Baeten, Javier Delgado      ***
   ***       and Francois Stricher          ***
   *** and any other of the 9! permutations ***
   ***   based on an original concept by    ***
   ***   Raphael Guerois and Luis Serrano   ***
   ********************************************

1 models read: EGFR_AlphaFold_Repair.pdb
Output: ./EGFR_AlphaFold_Repair.log

BackHbond       =               -669.24
SideHbond       =               -365.90
Energy_VdW      =               -1134.59
Electro         =               -47.46
Energy_SolvP    =               1498.79
E

In [ ]:
import pandas as pd
import os

# 1. Doğru dosya ismini kontrol edelim
# FoldX bazen dosya isminin başına 'Dif_' ekler.
possible_files = [f for f in os.listdir('.') if f.endswith('.fxout') and 'Dif' in f]

if not possible_files:
    print("HATA: FoldX sonuç dosyası (.fxout) bulunamadı. Lütfen BuildModel işleminin bittiğinden emin olun.")
else:
    filename = possible_files[0]
    print(f"Okunan dosya: {filename}")

    try:
        # Dosyayı önce ham olarak okuyup sütunları kontrol edelim
        # FoldX genelde 8 satır başlık (header) bilgisi verir
        df_check = pd.read_csv(filename, sep='\t', skiprows=8)

        # Sütun isimlerini temizleyelim (başındaki/sonundaki boşlukları silelim)
        df_check.columns = [c.strip() for c in df_check.columns]

        print(f"Bulunan sütunlar: {df_check.columns.tolist()}")

        # Sütunları dinamik olarak bulalım
        # İlk sütun genelde mutasyon adıdır (ismi 'Pdb', 'Mutant' veya '# Mutants' olabilir)
        mutant_col = df_check.columns[0]
        # Enerji farkı sütunu genelde 'total energy' veya 'Total' ismindedir
        energy_col = 'total energy' if 'total energy' in df_check.columns else df_check.columns[1]

        summary = df_check[[mutant_col, energy_col]].copy()
        summary.columns = ['Mutation', 'ddG (kcal/mol)']

        # Yorumlama fonksiyonu
        def interpret_ddg(val):
            try:
                val = float(val)
                if val > 1.5: return "Çok Kararsızlaştırıcı"
                if val > 0.5: return "Kararsızlaştırıcı"
                if val < -0.5: return "Stabilize Edici"
                return "Nötr"
            except:
                return "Hata"

        summary['Interpretation'] = summary['ddG (kcal/mol)'].apply(interpret_ddg)

        print("\n--- ANALİZ SONUÇLARI ---")
        print(summary.to_string(index=False))

        summary.to_csv('egfr_mutasyon_ozet.csv', index=False)
        print("\nSonuçlar 'egfr_mutasyon_ozet.csv' olarak kaydedildi.")

    except Exception as e:
        print(f"Bir hata oluştu: {e}")

Okunan dosya: Dif_EGFR_AlphaFold_Repair.fxout
Bulunan sütunlar: ['Pdb', 'total energy', 'Backbone Hbond', 'Sidechain Hbond', 'Van der Waals', 'Electrostatics', 'Solvation Polar', 'Solvation Hydrophobic', 'Van der Waals clashes', 'entropy sidechain', 'entropy mainchain', 'sloop_entropy', 'mloop_entropy', 'cis_bond', 'torsional clash', 'backbone clash', 'helix dipole', 'water bridge', 'disulfide', 'electrostatic kon', 'partial covalent bonds', 'energy Ionisation', 'Entropy Complex']

--- ANALİZ SONUÇLARI ---
                    Mutation  ddG (kcal/mol)        Interpretation
 EGFR_AlphaFold_Repair_1.pdb        1.560380 Çok Kararsızlaştırıcı
 EGFR_AlphaFold_Repair_2.pdb        0.020031                  Nötr
 EGFR_AlphaFold_Repair_3.pdb        0.257134                  Nötr
 EGFR_AlphaFold_Repair_4.pdb        0.474284                  Nötr
 EGFR_AlphaFold_Repair_5.pdb       -1.719400       Stabilize Edici
 EGFR_AlphaFold_Repair_6.pdb        4.702310 Çok Kararsızlaştırıcı
 EGFR_AlphaFold_Rep

In [ ]:
# 1. Gereksinimleri yükle
!apt-get update
!apt-get install -y build-essential cmake git

# 2. Kaynak kodu indir (eğer daha önce indirmediysen)
!git clone https://github.com/Discngine/fpocket.git

# 3. fpocket klasörüne git
%cd /content/fpocket

# 4. Derleme
!make clean
!make

Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,576 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,862 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [6,205 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [69.2 kB]
Get:14 htt

In [ ]:
# Vahşi Tip (WT)
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair.pdb
# Kritik Mutantlar (Örnek: 8. mutasyon GA598V ve 11. mutasyon LA858R)
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_8.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_11.pdb

***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 


In [ ]:
# Vahşi Tip (WT)
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair.pdb

# 17 Mutasyonun her biri için tek tek fpocket çalıştırılması
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_1.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_2.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_3.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_4.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_5.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_6.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_7.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_8.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_9.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_10.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_11.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_12.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_13.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_14.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_15.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_16.pdb
!/content/fpocket/bin/fpocket -f /content/EGFR_AlphaFold_Repair_17.pdb

***** POCKET HUNTING BEGINS ***** 
mkdir: cannot create directory ‘/content/EGFR_AlphaFold_Repair_out/pockets’: File exists
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
mkdir: cannot create directory ‘/content/EGFR_AlphaFold_Repair_8_out/pockets’: File exists
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
***** POCKET HUNTING ENDS ***** 
***** POCKET HUNTING BEGINS ***** 
mkdir: cannot create directory ‘/cont

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.display import display

# ======================================================
# 1. fpocket INFO DOSYASI OKUMA
# ======================================================

def parse_pocket_info(info_path):
    """
    fpocket *_info.txt dosyasından metrikleri güvenli şekilde çeker
    (tek cep / global pocket formatı)
    """
    metrics = {
        "score": np.nan,
        "druggability": np.nan,
        "volume": np.nan,
        "sasa": np.nan,
        "hydrophobicity": np.nan
    }

    try:
        with open(info_path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()

            def extract(pattern):
                m = re.search(pattern, content, re.IGNORECASE)
                return float(m.group(1)) if m else np.nan

            metrics["score"] = extract(r"Score\s*:\s*([-+]?\d*\.?\d+)")
            metrics["druggability"] = extract(r"Druggability Score\s*:\s*([-+]?\d*\.?\d+)")
            metrics["volume"] = extract(r"Volume\s*:\s*([-+]?\d*\.?\d+)")
            metrics["sasa"] = extract(r"SASA\s*:\s*([-+]?\d*\.?\d+)")
            metrics["hydrophobicity"] = extract(r"Hydrophobicity score\s*:\s*([-+]?\d*\.?\d+)")
    except Exception as e:
        print(f"HATA: {info_path} okunamadı -> {e}")

    return metrics


# ======================================================
# 2. fpocket ÇIKTILARINI TOPLA (RECURSIVE)
# ======================================================

def collect_fpocket_metrics(out_dir):
    """
    *_out klasörü altındaki TÜM *_info.txt dosyalarını bulur
    (pocket adı varsayımı YOK)
    """
    rows = []

    for root, _, files in os.walk(out_dir):
        for file in files:
            if file.endswith("_info.txt"):
                full_path = os.path.join(root, file)

                data = parse_pocket_info(full_path)
                data["source_file"] = file
                data["source_path"] = root
                rows.append(data)

    if not rows:
        return pd.DataFrame()

    return pd.DataFrame(rows)


# ======================================================
# 3. fpocket KLASÖRLERİ
# ======================================================

folders = {
    "WT": "/content/EGFR_AlphaFold_Repair_out",
    "GA598V": "/content/EGFR_AlphaFold_Repair_8_out",
    "LA858R": "/content/EGFR_AlphaFold_Repair_11_out"
}

results = {}

for name, path in folders.items():
    if os.path.exists(path):
        df = collect_fpocket_metrics(path)
        if not df.empty:
            results[name] = df
            print(f"✅ {name}: {len(df)} info dosyası okundu")
        else:
            print(f"⚠️ {name}: info dosyası bulunamadı")
    else:
        print(f"❌ {name}: dizin bulunamadı")

# ======================================================
# 4. PDS HESABI (WT DAHİL TABLO OLUŞTURMA)
# ======================================================

if "WT" in results:
    # 1. Vahşi Tip (WT) için en iyi cebi (referans) bul
    wt_df = results["WT"].sort_values("druggability", ascending=False)

    if not wt_df.empty:
        wt_best = wt_df.iloc[0]
        print(f"\n📍 Referans Cep (WT): {wt_best['source_file']} | Druggability: {wt_best['druggability']:.3f}")

        final_comparison = []

        # 2. Karşılaştırılacak tüm yapıları listele (WT'yi de ekliyoruz)
        target_structures = ["WT", "GA598V", "LA858R"]

        for name in target_structures:
            if name in results:
                # Yapının kendi içindeki en iyi cebini bul
                curr_df = results[name].sort_values("druggability", ascending=False)

                if not curr_df.empty:
                    curr_best = curr_df.iloc[0]

                    # WT referansına göre farkları hesapla
                    dvol = curr_best["volume"] - wt_best["volume"]
                    dsasa = curr_best["sasa"] - wt_best["sasa"]
                    dhyd = curr_best["hydrophobicity"] - wt_best["hydrophobicity"]

                    # PDS Formülü: |ΔVolume| + |ΔSASA| + |ΔHydrophobicity|
                    pds = abs(dvol) + abs(dsasa) + abs(dhyd)

                    final_comparison.append({
                        "Structure": name,
                        "Pocket_File": curr_best["source_file"],
                        "Druggability": round(curr_best["druggability"], 3),
                        "Volume": round(curr_best["volume"], 2),
                        "SASA": round(curr_best["sasa"], 2),
                        "dVolume": round(dvol, 2),
                        "dSASA": round(dsasa, 2),
                        "PDS": round(pds, 2)
                    })

        # 3. Sonuçları Tablo Olarak Göster
        if final_comparison:
            pds_df = pd.DataFrame(final_comparison)
            print("\n--- POCKET DEVIATION SCORE (PDS) ANALİZ SONUÇLARI (WT Referanslı) ---")
            display(pds_df)

            pds_df.to_csv("fpocket_pds_comprehensive_results.csv", index=False)
        else:
            print("❌ Karşılaştırma için veri bulunamadı.")
    else:
        print("❌ WT veri setinde geçerli bir cep bulunamadı.")

✅ WT: 1 info dosyası okundu
✅ GA598V: 1 info dosyası okundu
✅ LA858R: 1 info dosyası okundu

📍 Referans Cep (WT): EGFR_AlphaFold_Repair_info.txt | Druggability: 0.016

--- POCKET DEVIATION SCORE (PDS) ANALİZ SONUÇLARI (WT Referanslı) ---


,Structure,Pocket_File,Druggability,Volume,SASA,dVolume,dSASA,PDS
0,WT,EGFR_AlphaFold_Repair_info.txt,0.016,252.94,79.64,0.00,0.0,0.00
1,GA598V,EGFR_AlphaFold_Repair_8_info.txt,0.016,249.74,79.64,-3.21,0.0,3.21
2,LA858R,EGFR_AlphaFold_Repair_11_info.txt,0.016,255.38,79.64,2.44,0.0,2.44


In [ ]:
import os
import re
import pandas as pd
import numpy as np
from IPython.display import display

# ======================================================
# 1. fpocket INFO DOSYASI OKUMA
# ======================================================

def parse_pocket_info(info_path):
    """
    fpocket *_info.txt dosyasından metrikleri güvenli şekilde çeker
    (tek cep / global pocket formatı)
    """
    metrics = {
        "score": np.nan,
        "druggability": np.nan,
        "volume": np.nan,
        "sasa": np.nan,
        "hydrophobicity": np.nan
    }

    try:
        with open(info_path, "r", encoding="utf-8", errors="ignore") as f:
            content = f.read()

            def extract(pattern):
                m = re.search(pattern, content, re.IGNORECASE)
                return float(m.group(1)) if m else np.nan

            metrics["score"] = extract(r"Score\s*:\s*([-+]?\d*\.?\d+)")
            metrics["druggability"] = extract(r"Druggability Score\s*:\s*([-+]?\d*\.?\d+)")
            metrics["volume"] = extract(r"Volume\s*:\s*([-+]?\d*\.?\d+)")
            metrics["sasa"] = extract(r"SASA\s*:\s*([-+]?\d*\.?\d+)")
            metrics["hydrophobicity"] = extract(r"Hydrophobicity score\s*:\s*([-+]?\d*\.?\d+)")

    except Exception as e:
        print(f"HATA: {info_path} okunamadı -> {e}")

    return metrics


# ======================================================
# 2. TEK BİR _out KLASÖRÜNDEN INFO OKUMA
# ======================================================

def collect_fpocket_metrics(out_dir):
    """
    Verilen *_out klasörü altındaki TÜM *_info.txt dosyalarını okur.
    Dosya isimlerine dokunmaz.
    """
    rows = []

    for root, _, files in os.walk(out_dir):
        for file in files:
            if file.endswith("_info.txt"):
                full_path = os.path.join(root, file)

                data = parse_pocket_info(full_path)
                data["source_file"] = file
                data["source_path"] = root
                rows.append(data)

    if not rows:
        return pd.DataFrame()

    return pd.DataFrame(rows)


# ======================================================
# 3. ANA KLASÖRDEKİ TÜM _out KLASÖRLERİNİ TOPLA
# ======================================================

def collect_all_structures(parent_dir):
    """
    parent_dir içindeki tüm *_out klasörlerini otomatik bulur
    ve içlerindeki *_info.txt dosyalarını okur.
    """
    results = {}

    for item in sorted(os.listdir(parent_dir)):
        out_path = os.path.join(parent_dir, item)

        if os.path.isdir(out_path) and item.endswith("_out"):
            structure_name = item.replace("_out", "")
            df = collect_fpocket_metrics(out_path)

            if not df.empty:
                results[structure_name] = df
                print(f"✅ {structure_name}: info dosyası okundu")
            else:
                print(f"⚠️ {structure_name}: info dosyası bulunamadı")

    return results


# ======================================================
# 4. ANA KLASÖR YOLU (BURAYI SEN DOLDUR)
# ======================================================

parent_folder = "/content/EGFR_AlphaFold_mutation"   # <-- TÜM *_out klasörlerinin olduğu yer

results = collect_all_structures(parent_folder)


# ======================================================
# 5. WT REFERANSLI TOPLU PDS HESABI (Tam Sıralı)
# ======================================================

wt_key = "EGFR_AlphaFold_Repair"

if wt_key not in results:
    print(f"❌ HATA: '{wt_key}' bulunamadı.")
else:
    # WT referans cebini al
    wt_df = results[wt_key].sort_values("druggability", ascending=False)
    wt_best = wt_df.iloc[0]

    final_rows = []

    for name, df in results.items():
        curr_df = df.sort_values("druggability", ascending=False)
        curr_best = curr_df.iloc[0]

        dvol = curr_best["volume"] - wt_best["volume"]
        dsasa = curr_best["sasa"] - wt_best["sasa"]
        dhyd = curr_best["hydrophobicity"] - wt_best["hydrophobicity"]
        pds = abs(dvol) + abs(dsasa) + abs(dhyd)

        final_rows.append({
            "Structure": name,
            "Pocket_File": curr_best["source_file"],
            "Druggability": round(curr_best["druggability"], 3),
            "Volume": round(curr_best["volume"], 2),
            "SASA": round(curr_best["sasa"], 2),
            "dVolume": round(dvol, 2),
            "dSASA": round(dsasa, 2),
            "PDS": round(pds, 2)
        })

    pds_df = pd.DataFrame(final_rows)

    # --- DOĞAL SIRALAMA FONKSİYONU ---
    # Metin içindeki sayıları bulur ve sayısal olarak döndürür
    def natural_sort_key(text):
        return [int(c) if c.isdigit() else c.lower() for c in re.split(r'(\d+)', text)]

    # Önce WT yapısını ayırıp en başa koyalım, sonra kalanları sayısal sıralayalım
    wt_row = pds_df[pds_df['Structure'] == wt_key]
    mutant_rows = pds_df[pds_df['Structure'] != wt_key].copy()

    # Mutantları "Structure" ismindeki sayıya göre sırala (Mutant_1, Mutant_2... Mutant_10)
    mutant_rows['temp_sort'] = mutant_rows['Structure'].apply(natural_sort_key)
    mutant_rows = mutant_rows.sort_values('temp_sort').drop('temp_sort', axis=1)

    # WT'yi en başa ekleyip birleştir
    pds_df = pd.concat([wt_row, mutant_rows], ignore_index=True)

    print("\n--- PDS ANALİZİ: WT BAŞTA VE MUTANTLAR SIRALI ---")
    display(pds_df)

    pds_df.to_csv("fpocket_PDS_final_sorted.csv", index=False)

✅ EGFR_AlphaFold_Repair_10: info dosyası okundu
✅ EGFR_AlphaFold_Repair_11: info dosyası okundu
✅ EGFR_AlphaFold_Repair_12: info dosyası okundu
✅ EGFR_AlphaFold_Repair_13: info dosyası okundu
✅ EGFR_AlphaFold_Repair_14: info dosyası okundu
✅ EGFR_AlphaFold_Repair_15: info dosyası okundu
✅ EGFR_AlphaFold_Repair_16: info dosyası okundu
✅ EGFR_AlphaFold_Repair_17: info dosyası okundu
✅ EGFR_AlphaFold_Repair_1: info dosyası okundu
✅ EGFR_AlphaFold_Repair_2: info dosyası okundu
✅ EGFR_AlphaFold_Repair_3: info dosyası okundu
✅ EGFR_AlphaFold_Repair_4: info dosyası okundu
✅ EGFR_AlphaFold_Repair_5: info dosyası okundu
✅ EGFR_AlphaFold_Repair_6: info dosyası okundu
✅ EGFR_AlphaFold_Repair_7: info dosyası okundu
✅ EGFR_AlphaFold_Repair_8: info dosyası okundu
✅ EGFR_AlphaFold_Repair_9: info dosyası okundu
✅ EGFR_AlphaFold_Repair: info dosyası okundu

--- PDS ANALİZİ: WT BAŞTA VE MUTANTLAR SIRALI ---


,Structure,Pocket_File,Druggability,Volume,SASA,dVolume,dSASA,PDS
0,EGFR_AlphaFold_Repair,EGFR_AlphaFold_Repair_info.txt,0.016,252.94,79.64,0.00,0.00,0.00
1,EGFR_AlphaFold_Repair_1,EGFR_AlphaFold_Repair_1_info.txt,0.016,258.20,74.81,5.26,-4.83,10.09
2,EGFR_AlphaFold_Repair_2,EGFR_AlphaFold_Repair_2_info.txt,0.016,255.69,79.64,2.75,0.00,2.75
3,EGFR_AlphaFold_Repair_3,EGFR_AlphaFold_Repair_3_info.txt,0.016,255.77,79.64,2.82,0.00,2.82
4,EGFR_AlphaFold_Repair_4,EGFR_AlphaFold_Repair_4_info.txt,0.016,253.63,79.64,0.69,0.00,0.69
5,EGFR_AlphaFold_Repair_5,EGFR_AlphaFold_Repair_5_info.txt,0.013,252.83,78.37,-0.12,-1.27,1.39
6,EGFR_AlphaFold_Repair_6,EGFR_AlphaFold_Repair_6_info.txt,0.016,263.25,79.64,10.31,0.00,10.31
7,EGFR_AlphaFold_Repair_7,EGFR_AlphaFold_Repair_7_info.txt,0.016,262.64,79.64,9.70,0.00,9.70
8,EGFR_AlphaFold_Repair_8,EGFR_AlphaFold_Repair_8_info.txt,0.016,249.74,79.64,-3.21,0.00,3.21
9,EGFR_AlphaFold_Repair_9,EGFR_AlphaFold_Repair_9_info.txt,0.016,253.93,79.64,0.99,0.00,0.99
